In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

In [2]:
path = Path.home() / 'PycharmProjects/data_toolkit/nlp/transformers_learning/data'
df = pd.read_feather(path / 'facebook_comments.ftr')
df['label'] = df['sentiment'].map({'positive': 0, 'negative': 1})
df.drop(columns=['sentiment'], inplace=True)

df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)
df_train.head()

,content,label
319,Hùng Hùng,0
11095,Sale thì sale nhưng dịch dã thế này shipper ko...,1
61,"mọi người ơi cho em hỏi, trước em đặt đơn ảo b...",0
2673,bán ngoài 1 đơn bằng trên shopee bán 2 đơn :v,0
1922,"760600360 , 633321648 , 667661566 , 602130369 ...",1


In [3]:
train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)

dataset = DatasetDict()
dataset['train'] = train
dataset['test'] = test
dataset = dataset.rename_column('content', 'text')
dataset = dataset.remove_columns(['__index_level_0__'])

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7845
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3363
    })
})

In [4]:
dataset['train'][0:4]

{'text': ['Hùng Hùng',
  'Sale thì sale nhưng dịch dã thế này shipper ko qua lấy thì cũng như ko ạ 😢 210726U4DX00T8 đơn này của em đã 2 ngày rồi ko thấy shipper qua lấy, em đang lo ko biết có bị tính tỷ lệ giao hàng trễ ko 😢',
  'mọi người ơi cho em hỏi, trước em đặt đơn ảo bằng máy vps thì chỗ phí vận chuyển (không tính trợ giá)=0, mà giờ em nhờ các bạn đặt đơn ảo thì chỗ phí vận chuyển đó lại được cộng vào doanh thu của shop là sao ạ, em cảm ơn',
  'bán ngoài 1 đơn bằng trên shopee bán 2 đơn :v'],
 'label': [0, 1, 0, 0]}

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
# tokenizer.add_special_tokens({'cls_token':'[CLS]','sep_token':'[SEP]'})
# model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [6]:
def preprocess_function(examples):
    return tokenizer(examples['text'],
                     truncation=True,
                     max_length=120,
                     padding='max_length',
                     return_tensors='pt',
                     )

In [7]:
inputs = preprocess_function(dataset['train'][3])
tokenizer.decode(inputs["input_ids"][0])

'<s> bán ngoài 1 đơn bằng trên shopee bán 2 đơn :v </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [8]:
tokenize_train = dataset['train'].map(preprocess_function, batched=True)
tokenize_test = dataset['test'].map(preprocess_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [9]:
training_args = TrainingArguments(
    output_dir="./sentiment_phobert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenize_train,
    eval_dataset=tokenize_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Kevin\miniconda3\envs\nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7845
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2455
  Number of trainable parameters = 134999810


Step,Training Loss
500,0.422900
1000,0.303500
1500,0.223800
2000,0.180700


Saving model checkpoint to ./sentiment_phobert\checkpoint-500
Configuration saved in ./sentiment_phobert\checkpoint-500\config.json
Model weights saved in ./sentiment_phobert\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./sentiment_phobert\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./sentiment_phobert\checkpoint-500\special_tokens_map.json
added tokens file saved in ./sentiment_phobert\checkpoint-500\added_tokens.json
Saving model checkpoint to ./sentiment_phobert\checkpoint-1000
Configuration saved in ./sentiment_phobert\checkpoint-1000\config.json
Model weights saved in ./sentiment_phobert\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./sentiment_phobert\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ./sentiment_phobert\checkpoint-1000\special_tokens_map.json
added tokens file saved in ./sentiment_phobert\checkpoint-1000\added_tokens.json
Saving model checkpoint to ./sentiment_phobert\checkpoint-1500
Con

TrainOutput(global_step=2455, training_loss=0.25804647845794615, metrics={'train_runtime': 435.5828, 'train_samples_per_second': 90.052, 'train_steps_per_second': 5.636, 'total_flos': 2418874487460000.0, 'train_loss': 0.25804647845794615, 'epoch': 5.0})

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
trainer.push_to_hub()

C:\Users\Kevin\miniconda3\envs\nlp\lib\site-packages\huggingface_hub\repository.py:705: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  warnings.warn(


OSError: Tried to clone a repository in a non-empty folder that isn't a git repository. If you really want to do this, do it manually:
git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.